# Mouvement des Corps Rigides
## KIN-6839

### Rototrans dans biorbd

In [1]:
# Import needed packages
import biorbd    as brbd
import numpy     as np
import BiorbdViz as brbdv

In [6]:
# Load a simple model
b = brbdv.BiorbdViz(model_path="../data/2boxes.bioMod")
# Play with it
b.set_q(np.array([1,0,0,np.pi/2,0,0,0,1,0,0,0,0]))

Note : biorbd's interface uses Euler angles. Look at the .bioMod file to find the corresponding sequence. Inside biorbd, rototrans are used, as seen during the class. Let's play a bit with these notions. 

In [26]:
# Code a function which takes a rototrans and outputs 6D vector of configuration
# Hints : 
# brbd.RotoTrans.toEulerAngles => <function biorbd.biorbd.RotoTrans.toEulerAngles(rt, seq)>
# brbd.object.to_array() => numpy array of the biorbd object
# np.hstack, np.vstack => horizontally or vertically stacks numpy arrays

def rt_to_config(rt) :
    eul   = brbd.RotoTrans.toEulerAngles(rt,'xyz')
    trans = rt.to_array()[:3,3] 
    return np.hstack([eul.to_array(),trans])


Test the function.

In [34]:
rt = brbd.RotoTrans()
print(f"Identity transform : \n {rt.to_array()}")
print(f"Equivalent coordinates : \n {rt_to_config(rt)}")

Identity transform : 
 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
Equivalent coordinates : 
 [-0.  0. -0.  0.  0.  0.]


Now, let's play with our boxes.
Build a rototrans matrix "rpz" for Rot(z,90°).
Build a rototrans matrix "rny" for Rot(y,-90°).

In [42]:
#rpz = brbd.RotoTrans(np.matrix([[0,-1,0,0],[1,0,0,0],[0,0,1,0],[0,0,0,1]]))
brbd.Rotation(np.matrix([[0,-1,0,0],[1,0,0,0],[0,0,1,0],[0,0,0,1]]))
rny = np.array([[0,0,-1,0],[0,1,0,0],[1,0,0,0],[0,0,0,1]])
rpz_config = rt_to_config(rpz)
rny_config = rt_to_config(rny)
b.set_q(np.hstack([rpz_config,rny_config]))

TypeError: Wrong number or type of arguments for overloaded function 'new_Rotation'.
  Possible C/C++ prototypes are:
    biorbd::utils::Rotation::Rotation(Eigen::Matrix3d const &)
    biorbd::utils::Rotation::Rotation()
    biorbd::utils::Rotation::Rotation(biorbd::utils::Vector const &,biorbd::utils::String const &)
    biorbd::utils::Rotation::Rotation(RigidBodyDynamics::Math::SpatialTransform const &)


Test the product of rototrans, as seen in class. Visulalize the result in the viewer. Try to left and right multiply, discuss.

In [ ]:
###Code

### Rappels d'intégration numérique

Intégrer une fonction mathématique peut se faire analytiquement (formules connues, IPP, etc.) ou numériquement. Le principe de base de l'intégration numérique repose sur des appels à la fonction que l'on cherche à intégrer. 
Coder la méthode des rectangles (Euler).
Comparer à la valeur analytique connue en faisant varier le pas d'intégration.

In [90]:
import matplotlib.pyplot as plt
from time import time

xmin = 0
xmax = 100
nbx = 1000
nbi = nbx - 1 # nombre d'intervalles

x = np.linspace(xmin, xmax, nbx)
y = np.cos(x)
plt.plot(x,y,"bo-")

integrale = 0
t = time()
for i in range(nbi):
    integrale = integrale + y[i]*(x[i+1]-x[i])
    # dessin du rectangle
    x_rect = [x[i], x[i], x[i+1], x[i+1], x[i]] # abscisses des sommets
    y_rect = [0   , y[i], y[i]  , 0     , 0   ] # ordonnees des sommets
    plt.plot(x_rect, y_rect,"r")
    
print("temps nécessaire :", time() - t)
print("intégrale numérique =", integrale)
print("intégrale exacte = ",np.sin(xmax))

plt.show()

temps nécessaire : 1.6873178482055664
intégrale numérique = -0.49905180657145
intégrale exacte =  -0.5063656411097588


Coder la méthode des trapèzes (Euler).
Comparer à la valeur analytique connue en faisant varier le pas d'intégration.

In [91]:
nbx = 1000
nbi = nbx - 1 # nombre d'intervalles

x = np.linspace(xmin, xmax, nbx)
y = np.cos(x)
plt.plot(x,y,"bo-")

integrale = 0
t = time()
for i in range(nbi):
    integrale = integrale + (y[i+1]+y[i])*(x[i+1]-x[i])/2
    # dessin du rectangle
    x_rect = [x[i], x[i], x[i+1], x[i+1], x[i]] # abscisses des sommets
    y_rect = [0   , y[i], y[i+1]  , 0     , 0   ] # ordonnees des sommets
    plt.plot(x_rect, y_rect,"r")
    
print("temps nécessaire :", time() - t)
print("intégrale numérique =", integrale)
print("intégrale exacte = ",np.sin(xmax))

plt.show()

temps nécessaire : 2.4084010124206543
intégrale numérique = -0.5059427539043958
intégrale exacte =  -0.5063656411097588


Lire d'autre méthodes d'intégration (Simpson, pôlynomes de Lagrange).
Utiliser l'intégration RK45 de scipy et comparer aux méthodes précédentes.

In [92]:
from scipy import integrate

def mycos(t,y):
    return np.cos(t)

t = time()
sol = integrate.solve_ivp(mycos, [xmin, xmax], [0], method='RK45')

print("temps nécessaire :", time() - t)
print("intégrale numérique =", -0.5060210157655309)
print("intégrale exacte = ",np.sin(xmax))


temps nécessaire : 0.00672459602355957
intégrale numérique = -0.5060210157655309
intégrale exacte =  -0.5063656411097588


### Retour aux corps rigides 

Intégrer une vitesse de rotation spatiale avec RK45. 
Comparer à la formule de Rodrigues.
Tracez l'erreur en degrés en fonction de la durée d'intégration.
Visualisez le résultat.

In [185]:
import math as math
from scipy import interpolate

T = 10 # Durée de la simulation
w = [0,0,1] # Axe de la rotation

def so3(x):
    return np.array([[0, -x[2], x[1]],
                     [x[2], 0, -x[0]],
                     [-x[1], x[0], 0]])

def diff_R(t,R,w):
    dot_R = so3(w).dot(np.reshape(R,(3,3)))
    return np.reshape(dot_R,9)


func_ode = lambda t, R : diff_R(t,R,w)

R_0 = np.reshape(np.eye(3),9)
sol = integrate.solve_ivp(func_ode, [0, T], R_0, method='RK45')
sol_interp = interpolate.interp1d(sol.t,sol.y,kind='cubic')

print("intégrale numérique =", np.reshape(sol.y[:,-1],(3,3)))

def rodrigues(w,T):
    return np.eye(3)+np.sin(T)*so3(w) + (1-np.cos(T))*so3(w).dot(so3(w))
int_rod = rodrigues(w,T)
print("intégrale exacte = ", int_rod)

def rot_to_eul(rot):
    psi   = math.atan2(rot[0,0],-rot[1,0])
    theta = math.acos(max(-1, min(1, rot[2,2])))
    phi   = math.atan2(rot[2,0],rot[2,1])
    return np.array([psi,theta,phi])

eul_int = []
interp_ratio = 1000
t_interp = np.linspace(0,T,interp_ratio)
for t in t_interp :
    eul_int += [rot_to_eul(np.reshape(sol_interp(t),(3,3)))]
plt.plot(t_interp,eul_int)


Ts = np.linspace(0,10,num=100)
err = []
err_eul = []
for T in Ts :
    sol = integrate.solve_ivp(func_ode, [0, T], R_0, method='RK45')
    int_rod = rodrigues(w,T)
    err += [np.linalg.norm(np.reshape(sol.y[:,-1],(3,3))-int_rod)]
    err_eul += [rot_to_eul(np.reshape(sol.y[:,-1],(3,3))) - rot_to_eul(int_rod)]
#plt.plot(Ts,err)
plt.figure()
plt.plot(Ts,err_eul)
plt.show()

intégrale numérique = [[-0.83695968  0.54807877  0.        ]
 [-0.54807877 -0.83695968  0.        ]
 [ 0.          0.          1.        ]]
intégrale exacte =  [[-0.83907153  0.54402111  0.        ]
 [-0.54402111 -0.83907153  0.        ]
 [ 0.          0.          1.        ]]
